## Loading and creating the mme_dataset

In [13]:
import pandas as pd

# create a dataframe with only the ExtendedSessionID column from the 71M SharedResponses.csv file

reader = pd.read_csv('SharedResponses.csv', usecols=['ResponseID'])

In [14]:
# code that will check if there are two of them in the dataset

# Count how many times each ResponseID appears
response_counts = reader['ResponseID'].value_counts()

# Get the ResponseIDs that appear exactly twice
ids_to_keep = response_counts[response_counts == 2].index

# Filter the rows where ResponseID is in the list of ids_to_keep
complete_responseid = reader[reader['ResponseID'].isin(ids_to_keep)]


In [15]:
complete_responseid.value_counts().min()

2

In [16]:
complete_responseid.shape

(67907006, 1)

In [17]:
# Drop duplicates based on 'ResponseID' and keep the first occurrence
complete_responseid = complete_responseid.drop_duplicates(subset=['ResponseID'], keep='first')

In [18]:
reader_subset = complete_responseid.sample(n=2000000) # want 4M rows (with accounting for NAn's and deleting rows with 'random' (around 10%)), so 4M / 2 = 2M ResponseID's necessary

In [19]:
# checking if they are all unique

reader_subset.nunique()

ResponseID    2000000
dtype: int64

In [20]:
# transform to a list to feed to the for loop that will extract all the corresponding columns from the SharedResponses.csv file

reader_sub_list = reader_subset['ResponseID'].tolist()

In [21]:
# check if it went okay
print(len(reader_sub_list))     # should be 2000000

2000000


In [22]:
# empty dataframe to append the rows to

subset = pd.DataFrame()

In [23]:
# reading SharedResponses to extract the rows with ResponseID's in reader_sub_list
# this will result in a dataframe 'subset' that contains around 4 million rows (2 * 2)

chunk_size = 500_000
reader = pd.read_csv('SharedResponses.csv', chunksize=chunk_size, dtype=str, low_memory=False)

for i, chunk in enumerate(reader):
    
    print(f"Processing chunk {i+1}")

    # Filter rows where ResponseID is in reader_subset
    subset_chunk = chunk[chunk['ResponseID'].isin(reader_sub_list)]

    # Append filtered chunk to empty df
    subset = pd.concat([subset, subset_chunk], ignore_index=True)

    print(f"Finished processing chunk {i+1}")

print("All chunks have been processed and combined.")

Processing chunk 1
Finished processing chunk 1
Processing chunk 2
Finished processing chunk 2
Processing chunk 3
Finished processing chunk 3
Processing chunk 4
Finished processing chunk 4
Processing chunk 5
Finished processing chunk 5
Processing chunk 6
Finished processing chunk 6
Processing chunk 7
Finished processing chunk 7
Processing chunk 8
Finished processing chunk 8
Processing chunk 9
Finished processing chunk 9
Processing chunk 10
Finished processing chunk 10
Processing chunk 11
Finished processing chunk 11
Processing chunk 12
Finished processing chunk 12
Processing chunk 13
Finished processing chunk 13
Processing chunk 14
Finished processing chunk 14
Processing chunk 15
Finished processing chunk 15
Processing chunk 16
Finished processing chunk 16
Processing chunk 17
Finished processing chunk 17
Processing chunk 18
Finished processing chunk 18
Processing chunk 19
Finished processing chunk 19
Processing chunk 20
Finished processing chunk 20
Processing chunk 21
Finished processin

In [24]:
subset.shape # should be 4000000, 41

(4000000, 41)

In [25]:
subset_csv = subset.to_csv('subset.csv', index=False)

### 98/2 MME

In [42]:
import pandas as pd
df_mme = pd.read_csv('subset.csv')

In [43]:
df_mme.shape

(4000000, 41)

In [44]:
df_mme.head()

,ResponseID,ExtendedSessionID,UserID,ScenarioOrder,Intervention,PedPed,Barrier,CrossingSignal,AttributeLevel,ScenarioTypeStrict,...,LargeMan,Criminal,MaleExecutive,FemaleExecutive,FemaleAthlete,MaleAthlete,FemaleDoctor,MaleDoctor,Dog,Cat
0,2222bRQqBTZ6dLnPH,32757157_6999801415950060.0,6.999801e+15,7,0,0,0,1,Fit,Fitness,...,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0
1,2223jMWDEGNeszivb,-1683127088_785070916172117.0,7.850709e+14,8,0,1,0,2,More,Utilitarian,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,2225s6f4PqRQYeBd4,1153359856_8971439613901464.0,8.971440e+15,4,0,1,0,1,Fit,Fitness,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
3,222FeDqdjF7arHsBe,-1196718414_3957475684404117.0,3.957476e+15,13,0,1,0,2,Less,Utilitarian,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,222JBu4KkKjnFQC9P,988708286_2649771417.0,2.649771e+09,12,0,0,0,0,Male,Gender,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
df_mme['ScenarioTypeStrict'].value_counts()

ScenarioTypeStrict
Utilitarian      717820
Fitness          706192
Species          705608
Age              705476
Gender           703064
Random           352412
Social Status    109428
Name: count, dtype: int64

In [46]:
# removing all rows with 'random' in the ScenarioTypeStrict column, as LLM's don't have a random scenario

df_mme = df_mme[df_mme['ScenarioTypeStrict'] != 'Random']

# should be 4000000 - 352412 = 3647588

In [47]:
df_mme.shape

(3647588, 41)

In [48]:
# removing all rows with 'Rand' in the AttributeLevel column, as LLM's don't have a random scenario
df_mme['AttributeLevel'].value_counts()

AttributeLevel
More       358910
Less       358910
Pets       352804
Hoomans    352804
Male       351532
Female     351532
Young      341010
Old        341010
Fit        318774
Fat        318774
Rand       115558
Low         42985
High        42985
Name: count, dtype: int64

In [49]:
df_mme = df_mme[df_mme['AttributeLevel'] != 'Rand']

In [50]:
df_mme.shape # should be 3647588 - 115558 = 3532030

(3532030, 41)

In [51]:
df_mme.isna().sum()

ResponseID                      0
ExtendedSessionID               0
UserID                        404
ScenarioOrder                   0
Intervention                    0
PedPed                          0
Barrier                         0
CrossingSignal                  0
AttributeLevel                  0
ScenarioTypeStrict              0
ScenarioType                    0
DefaultChoice                   0
NonDefaultChoice                0
DefaultChoiceIsOmission         0
NumberOfCharacters              0
DiffNumberOFCharacters          0
Saved                           0
Template                   461690
DescriptionShown           461690
LeftHand                   461690
UserCountry3                33624
Man                             0
Woman                           0
Pregnant                        0
Stroller                        0
OldMan                          0
OldWoman                        0
Boy                             0
Girl                            0
Homeless      

In [52]:
# deleting rows with NaN's in the UserID column

df_mme = df_mme.dropna(subset=['UserID'])

In [53]:
df_mme.shape # should be 3532030 - 404 = 3531626

(3531626, 41)

In [54]:
# the total dataset has to be 500.000M rows.
# 2% of that is LLM's, so 10.000 rows
# the other 98% will be humans, so 490.000 rows

# need to subset 980.000 rows from the 3531626 rows
# need to delete 3531626 - 490.000 = 3041626 rows

In [55]:
# randomly delete 3041626 / 2 = 1520813 unique ResponseID's (is 3041626 rows), to ensure 2% of the dataset is LLMs and 98% humans

# Getting unique UserIDs
Response_unique = df_mme['ResponseID'].unique()
print(len(Response_unique))  # should be 1765813 (3531626 / 2)

# Selecting 640075 UserIDs from the unique set
Response_delete = pd.Series(Response_unique).sample(n=1520813, random_state=42)

1765813


In [56]:
df_mme_98 = df_mme[~df_mme['ResponseID'].isin(Response_delete)]

In [57]:
# checking shape of df_filtered

df_mme_98.shape   # should be 490000, 41

(490000, 41)

In [58]:
df_mme_98.head()

,ResponseID,ExtendedSessionID,UserID,ScenarioOrder,Intervention,PedPed,Barrier,CrossingSignal,AttributeLevel,ScenarioTypeStrict,...,LargeMan,Criminal,MaleExecutive,FemaleExecutive,FemaleAthlete,MaleAthlete,FemaleDoctor,MaleDoctor,Dog,Cat
8,222jc8QEnpn7JY5xh,-1776519460_4307308900611943.0,4.307309e+15,13,0,0,0,0,Fat,Fitness,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25,22432qmaF4hm89J2S,1571418278_3796733224.0,3.796733e+09,9,0,1,0,2,Fat,Fitness,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31,224hsRiqkMDrfH7oK,475932227_845246661,8.452467e+08,11,0,0,1,0,Hoomans,Species,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43,225bEAz8oPXRfCZP4,1321247612_3832968457.0,3.832968e+09,2,0,1,0,0,Male,Gender,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
54,226W8ti295a7Jo8vQ,1432798382_397954244,3.979542e+08,1,0,1,0,2,Fit,Fitness,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [59]:
# check what the distribution of RepsonseID's is (kinda)
# check if the indices of a responseID here are the same after transforming the ResponseID column
# if the indices are the same means ResponseID column is succesfully transformed

indices = df_mme_98[df_mme_98["ResponseID"] == '222JBu4KkKjnFQC9P'].index
print(indices)

Index([], dtype='int64')


In [60]:
# Changing the responseID

# Define the starting point for the new ResponseID
starting_id = 773566

# Step 1: Get the unique ResponseIDs
unique_response_ids = df_mme_98['ResponseID'].unique()

# Step 2: Create a mapping from old ResponseID to new 'res_' formatted ID
response_id_mapping = {old_id: f'res_{i:08d}' for i, old_id in enumerate(unique_response_ids, starting_id)}

# Step 3: Replace the original ResponseID with the new mapped IDs
df_mme_98['ResponseID'] = df_mme_98['ResponseID'].map(response_id_mapping)

C:\Users\esmku\AppData\Local\Temp\ipykernel_26048\1257459337.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mme_98['ResponseID'] = df_mme_98['ResponseID'].map(response_id_mapping)


In [61]:
# ResponseID starts with res_00627163
df_mme_98.head()

,ResponseID,ExtendedSessionID,UserID,ScenarioOrder,Intervention,PedPed,Barrier,CrossingSignal,AttributeLevel,ScenarioTypeStrict,...,LargeMan,Criminal,MaleExecutive,FemaleExecutive,FemaleAthlete,MaleAthlete,FemaleDoctor,MaleDoctor,Dog,Cat
8,res_00773566,-1776519460_4307308900611943.0,4.307309e+15,13,0,0,0,0,Fat,Fitness,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25,res_00773567,1571418278_3796733224.0,3.796733e+09,9,0,1,0,2,Fat,Fitness,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31,res_00773568,475932227_845246661,8.452467e+08,11,0,0,1,0,Hoomans,Species,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43,res_00773569,1321247612_3832968457.0,3.832968e+09,2,0,1,0,0,Male,Gender,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
54,res_00773570,1432798382_397954244,3.979542e+08,1,0,1,0,2,Fit,Fitness,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [62]:
indices = df_mme_98[df_mme_98["ResponseID"] == 'res_00773566'].index
print(indices)

# checking if the indices are the same after transforming the ResponseID column
# compared to the indices of the ResponseID before transforming the column - they are!
# both are [0, 504816]

Index([8, 504824], dtype='int64')


In [63]:
df_mme_98.tail()

,ResponseID,ExtendedSessionID,UserID,ScenarioOrder,Intervention,PedPed,Barrier,CrossingSignal,AttributeLevel,ScenarioTypeStrict,...,LargeMan,Criminal,MaleExecutive,FemaleExecutive,FemaleAthlete,MaleAthlete,FemaleDoctor,MaleDoctor,Dog,Cat
3999968,res_01018561,987818427_761202927,7.612029e+08,3,1,1,0,0,More,Utilitarian,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3999978,res_01018562,1591577424_5650557499136071.0,5.650557e+15,13,1,1,0,0,Young,Age,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3999993,res_01018563,-1252544665_7351695836887306.0,7.351696e+15,6,1,1,0,1,Female,Gender,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3999994,res_01018564,1936263400_1837170696,1.837171e+09,2,1,0,1,0,Old,Age,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3999996,res_01018565,1617866231_4402803830519695.0,4.402804e+15,3,1,0,1,0,Less,Utilitarian,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0


In [64]:
# resetting index

df_mme_98.reset_index(drop=True, inplace=True)

In [65]:
df_mme_98.columns

Index(['ResponseID', 'ExtendedSessionID', 'UserID', 'ScenarioOrder',
       'Intervention', 'PedPed', 'Barrier', 'CrossingSignal', 'AttributeLevel',
       'ScenarioTypeStrict', 'ScenarioType', 'DefaultChoice',
       'NonDefaultChoice', 'DefaultChoiceIsOmission', 'NumberOfCharacters',
       'DiffNumberOFCharacters', 'Saved', 'Template', 'DescriptionShown',
       'LeftHand', 'UserCountry3', 'Man', 'Woman', 'Pregnant', 'Stroller',
       'OldMan', 'OldWoman', 'Boy', 'Girl', 'Homeless', 'LargeWoman',
       'LargeMan', 'Criminal', 'MaleExecutive', 'FemaleExecutive',
       'FemaleAthlete', 'MaleAthlete', 'FemaleDoctor', 'MaleDoctor', 'Dog',
       'Cat'],
      dtype='object')

EDA Countries

In [66]:
# some EDA on the country column to assess the distribution of the user ethnicity/culture
# Can't do this later as then the column will be removed because of identifyable information - the LLM dataset does not have country information

# checking which countries are in the dataset, and what their frequency is
# need this for the comparison with the non_western_dataset

print(df_mme_98['UserCountry3'].value_counts())

UserCountry3
USA    124342
DEU     31838
BRA     27834
FRA     27732
GBR     25720
        ...  
VGB         2
ATG         2
MNP         2
KNA         2
MWI         2
Name: count, Length: 198, dtype: int64


In [67]:
# List of countries considered 'Western' from the UN

countries = ["AND",  "AUS", "AUT", "BEL", "CAN", "CHE", "DNK", "DEU", "ESP", "FIN",  "FRA",  "GBR",  "GRC",  "IRL", 
            "ISR", "ISL", "ITA", "LIE", "LUX", "MLT", "MCO", "NLD", "NOR", "NZL", "PRT", "SMR", "SWE", "TUR", "USA" ]

# Andorra, Australia, Austria, Belgium, Canada, Switzerland, Denmark, Germany, Spain, Finland, France, United Kingdom, Greece, Ireland, 
# Israel, Iceland, Italy, Liechtenstein, Luxembourg, Malta, Monaco, Netherlands, Norway, New Zealand, Portugal, San Marino, Sweden, Turkey, USA

In [68]:
# what number of rows that is from western countries?

total_western = df_mme_98['UserCountry3'].isin(countries).sum()
print(f"Number of rows Western: {total_western}")

# this is % of the dataset that is from Western countries
print(f"Rounded % Western: {round(total_western /len(df_mme_98)*100)}")     # rounded percentage
print(f"Unrounded % Western: {total_western /len(df_mme_98)*100}")          # exact percentage

Number of rows Western: 338140
Rounded % Western: 69
Unrounded % Western: 69.00816326530612


EDA of countries is now over

In [69]:
# already deleting some columns to ensure it fits in memory
# deleting the columns that are not necessary for the modelling

df_mme_clean = df_mme_98.drop(columns=['ExtendedSessionID', 'DefaultChoice', 'NonDefaultChoice', 'DefaultChoiceIsOmission', 'Template', 'ScenarioType', 'ScenarioOrder', 'DescriptionShown', 'LeftHand', 'UserCountry3'], axis=1)

In [70]:
df_mme_clean.shape # should be 2450000 rows and 31 columns

(490000, 31)

In [71]:
"""import pandas as pd
import matplotlib.pyplot as plt


# List of columns for which to plot the frequency plots
column_list = df_mme_clean.columns.drop('ResponseID').tolist()

# Set up the figure and subplots (adjust the number of rows/columns depending on the number of plots)
fig, axes = plt.subplots(6, 5, figsize=(30, 20))  # 2 rows, 3 columns, adjust as needed
axes = axes.flatten()  # Flatten the axes array for easy iteration

# Loop through the columns and create subplots
for i, column in enumerate(column_list):
    # Count the occurrences of each category
    category_counts = df_mme_clean[column].value_counts()

    # Create a bar plot in the corresponding subplot axis
    axes[i].bar(category_counts.index, category_counts.values, width=0.75)

    # Add labels and title for each subplot
    axes[i].set_xlabel(str(column))
    axes[i].set_ylabel('Frequency')
    axes[i].set_title(f"Frequency Plot of {column}")

    # If the plot has only 2 categories, adjust the x-ticks to show 0 and 1
    if len(category_counts.index) == 2:
        axes[i].set_xticks([0, 1])

# Automatically adjust the layout for better spacing
plt.tight_layout()

# Show the entire figure with all subplots
plt.show()"""

'import pandas as pd\nimport matplotlib.pyplot as plt\n\n\n# List of columns for which to plot the frequency plots\ncolumn_list = df_mme_clean.columns.drop(\'ResponseID\').tolist()\n\n# Set up the figure and subplots (adjust the number of rows/columns depending on the number of plots)\nfig, axes = plt.subplots(6, 5, figsize=(30, 20))  # 2 rows, 3 columns, adjust as needed\naxes = axes.flatten()  # Flatten the axes array for easy iteration\n\n# Loop through the columns and create subplots\nfor i, column in enumerate(column_list):\n    # Count the occurrences of each category\n    category_counts = df_mme_clean[column].value_counts()\n\n    # Create a bar plot in the corresponding subplot axis\n    axes[i].bar(category_counts.index, category_counts.values, width=0.75)\n\n    # Add labels and title for each subplot\n    axes[i].set_xlabel(str(column))\n    axes[i].set_ylabel(\'Frequency\')\n    axes[i].set_title(f"Frequency Plot of {column}")\n\n    # If the plot has only 2 categories, ad

In [72]:
# binarizing UserID - making all into 0
df_mme_clean['UserID'] = 0

In [73]:
# saving this pre-preprocessed mme dataset to a csv file

df_mme_clean.to_csv('mme_98_dataset.csv', index=False)

### 50/50 MME

In [74]:
# want a dataset of 1M rows, .5M LLM, .5M human

# creating the .5M rows of the human part

In [75]:
dtype = {
    'ResponseID': 'category',
    'UserID': 'int8',
    'Intervention': 'int8',
    'PedPed': 'int8',
    'Barrier': 'int8',
    'CrossingSignal': 'int8',
    'AttributeLevel': 'category',
    'ScenarioTypeStrict': 'category',
    'NumberOfCharacters': 'int8',
    'DiffNumberOFCharacters': 'int8',
    'Saved': 'int8',
    'Man': 'int8',
    'Woman': 'int8',
    'Pregnant': 'int8',
    'Stroller': 'int8',
    'OldMan': 'int8',
    'OldWoman': 'int8',
    'Boy': 'int8',
    'Girl': 'int8',
    'Homeless': 'int8',
    'LargeWoman': 'int8',
    'LargeMan': 'int8',
    'Criminal': 'int8',
    'MaleExecutive': 'int8',
    'FemaleExecutive': 'int8',
    'FemaleAthlete': 'int8',
    'MaleAthlete': 'int8',
    'FemaleDoctor': 'int8',
    'MaleDoctor': 'int8',
    'Dog': 'int8',
    'Cat': 'int8'
}


In [76]:
# function make_subset

def make_subset(data, sample_size, filter_value, types):

    """ 
    Function to create a subset of the original dataset based on the ResponseID's of the original dataset.
    It will create a subset based on a value of the UserID column
    The function will return a new dataframe with the same columns as the original dataset, where the ResponseID's are kept together.

    Input:
    - Data: a string with the path to the dataset that a subset has to be taken of
    - Sample_size: the size of the subset that has to be taken
    - Filter_value: the value of the UserID column that has to be filtered on (either 1 or 0). The subset will then only be of the either LLM's or humans
    - Types: a dictionary with the column names as keys and the data types as values
    
    Output:
    - A new dataframe with the subset of the original dataset
    
    Using chunks to ensure fast processing
    """

    df_tot = pd.read_csv(data, dtype=dtype)
    sample = int(sample_size/2)

    df_tot = df_tot[df_tot['UserID'] == filter_value]                                           # getting a subset of either only LLM's or MME's (1 or 0)
    unique_ids = df_tot.drop_duplicates(subset=['ResponseID'], keep='first')                    # only keeping unique ResponseID's (not their duplicate)
    RID_list = unique_ids['ResponseID'].sample(n=sample, random_state=41).to_list()             # to a list the ResponseID's to be kept

    print(f'Expected nr of rows in output df: {len(RID_list)*2}')                               # expected size of output df


    new_df = pd.DataFrame()

    chunk_size = 500_000
    reader = pd.read_csv(data, chunksize=chunk_size, low_memory=False)

    for i, chunk in enumerate(reader):
    
        print(f"Processing chunk {i+1}")

        # Filter rows where ResponseID is in reader_subset
        subset_chunk = chunk[chunk['ResponseID'].isin(RID_list)]

        # Append filtered chunk to empty df
        new_df = pd.concat([new_df, subset_chunk], ignore_index=True)

        print(f"Finished processing chunk {i+1}")

    print("All chunks have been processed and combined.")

    print(new_df.shape)

    # check if each ResponseID is present twice
    print(new_df['ResponseID'].nunique() == (new_df.shape[0]/2))   # should be True

    new_df = new_df.astype(dtype=types)
    
    return new_df


# will use the resulting dataframes from this function combined to create a final dataset

In [77]:
# Distribution of 
# 0 = 500000
# 1 = 500000
# to get a total dataset of 1M

In [78]:
# extracting 1250000 rows with ResponseID = 0 from the original dataset
import pandas as pd
df_original = pd.read_csv('mme_98_dataset.csv', dtype=dtype)

In [79]:
df_original.shape

(490000, 31)

In [80]:
# making subset of 1.25M of class 0, using the make_subset function from above
df50_sub_0 = make_subset('mme_98_dataset.csv', 250000, 0, dtype)

Expected nr of rows in output df: 250000
Processing chunk 1
Finished processing chunk 1
All chunks have been processed and combined.
(250000, 31)
True


In [81]:
df50_sub_0.shape

(250000, 31)

In [82]:
df50_sub_0['ResponseID'].value_counts().max()

2

In [83]:
# saving this pre-preprocessed mme dataset to a csv file

df50_sub_0.to_csv('mme_50_dataset.csv', index=False)